In [1]:
import lipinet.analyse  # Import the module
import graph_tool as gt
import importlib

# Reload the module to ensure changes are picked up
importlib.reload(lipinet.analyse)

# Now you can use your function after reloading the module
from lipinet.analyse import propagate_categorical_property

In [7]:
g = gt.Graph(directed=True)

# Create vertex property for reactions
reactions = g.new_vertex_property("object")

# Add nodes
nodes = [g.add_vertex() for _ in range(12)]  # Nodes 0 to 11

# Assign reactions
reactions[nodes[0]] = ["R0"]                 # Root node
reactions[nodes[1]] = ["R1"]                 # Level 1 node
reactions[nodes[2]] = ["R2"]                 # Measured node
reactions[nodes[3]] = ["R3", "R4"]           # Child of measured node
reactions[nodes[4]] = ["R4"]                 # Child of measured node (shared reaction R4)
reactions[nodes[5]] = ["R5"]                 # Child of node 3
reactions[nodes[6]] = ["R6"]                 # Child of node 3
reactions[nodes[7]] = ["R5"]                 # Child of node 4 (shared reaction R5)
reactions[nodes[8]] = ["R7", "R8"]           # Child of node 4
reactions[nodes[9]] = ["R9", "R10"]          # Child of node 5
reactions[nodes[10]] = ["R9"]                # Child of node 7 (shared reaction R9)
reactions[nodes[11]] = ["R11"]               # Child of node 8

# Add edges to represent hierarchy
g.add_edge(nodes[0], nodes[1])   # Root -> Level 1
g.add_edge(nodes[1], nodes[2])   # Level 1 -> Measured Node
g.add_edge(nodes[2], nodes[3])   # Measured Node -> Node 3
g.add_edge(nodes[2], nodes[4])   # Measured Node -> Node 4
g.add_edge(nodes[3], nodes[5])   # Node 3 -> Node 5
g.add_edge(nodes[3], nodes[6])   # Node 3 -> Node 6
g.add_edge(nodes[4], nodes[7])   # Node 4 -> Node 7
g.add_edge(nodes[4], nodes[8])   # Node 4 -> Node 8
g.add_edge(nodes[5], nodes[9])   # Node 5 -> Node 9
g.add_edge(nodes[7], nodes[10])  # Node 7 -> Node 10
g.add_edge(nodes[8], nodes[11])  # Node 8 -> Node 11

# Use the categorical property propagation function
accumulated_categories_downstream = propagate_categorical_property(g, nodes[2], reactions, verbose=True, showtotal=True)

Propagating from node 2 in direction 'downstream'
Processing node 2, with weight 1.0 and inherited categories {}
Processing node 3, with weight 0.5 and inherited categories {'R2': 'measured_node'}
Processing node 5, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R3': 'downstream', 'R4': 'downstream'}
Processing node 9, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R3': 'downstream', 'R4': 'downstream', 'R5': 'downstream'}
Processing node 6, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R3': 'downstream', 'R4': 'downstream'}
Processing node 4, with weight 0.5 and inherited categories {'R2': 'measured_node'}
Processing node 7, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R4': 'downstream'}
Processing node 10, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R4': 'downstream', 'R5': 'downstream'}
Processing node 8, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R4': 'd

In [8]:
# try just upstream propagation
accumulated_categories_upstream = propagate_categorical_property(g, nodes[2], reactions, 
                               verbose=True, showtotal=True,
                               direction='upstream')

Propagating from node 2 in direction 'upstream'
Processing node 2, with weight 1.0 and inherited categories {}
Processing node 1, with weight 1.0 and inherited categories {'R2': 'measured_node'}
Processing node 0, with weight 1.0 and inherited categories {'R2': 'measured_node', 'R1': 'upstream'}
Accumulated Categories at Terminal Nodes (from node 2):

Categories from upstream:
  Category R1: Accumulated Weight 1.0
  Category R0: Accumulated Weight 1.0

Categories from measured node:
  Category R2: Accumulated Weight 1.0

Categories from downstream:


In [9]:
# try both upstream and downstream propagation
accumulated_categories_both = propagate_categorical_property(g, nodes[2], reactions, 
                               verbose=True, showtotal=True,
                               direction='both')

Propagating from node 2 in direction 'both'
Processing node 2, with weight 1.0 and inherited categories {}
Processing node 3, with weight 0.5 and inherited categories {'R2': 'measured_node'}
Processing node 5, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R3': 'downstream', 'R4': 'downstream'}
Processing node 9, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R3': 'downstream', 'R4': 'downstream', 'R5': 'downstream'}
Processing node 6, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R3': 'downstream', 'R4': 'downstream'}
Processing node 4, with weight 0.5 and inherited categories {'R2': 'measured_node'}
Processing node 7, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R4': 'downstream'}
Processing node 10, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R4': 'downstream', 'R5': 'downstream'}
Processing node 8, with weight 0.25 and inherited categories {'R2': 'measured_node', 'R4': 'downstr

In [10]:
both

{'upstream': {'R1': 1.0, 'R0': 1.0},
 'measured_node': {'R2': 1.0},
 'downstream': {'R3': 0.5,
  'R4': 1.0,
  'R5': 0.5,
  'R9': 0.5,
  'R10': 0.25,
  'R6': 0.25,
  'R7': 0.25,
  'R8': 0.25,
  'R11': 0.25}}